In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier

In [104]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv') #угадать, кто выжил

In [105]:
pwd() #где найти наши файлы

'C:\\Users\\Максим\\Documents'

In [106]:
train.describe()

C:\Users\Максим\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [107]:
test.describe()

C:\Users\Максим\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,NaN,0.000000,0.000000,NaN
50%,1100.500000,3.000000,NaN,0.000000,0.000000,NaN
75%,1204.750000,3.000000,NaN,1.000000,0.000000,NaN
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [108]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [109]:
all_df = (train, test) #просто список двух датасетов 

#### Что важно?
1. Создание переменных
2. Заполнение пропусов (мы заполним медианой)
3. Параметры алгоритма (энтропия, индекс Джини)

In [149]:
big_df = pd.concat(all_df)  #test прикрепили к train снизу

### Заполняем NaN

In [111]:
age_median = big_df["Age"].median()
age_median

28.0

In [112]:
fare_median = big_df["Fare"].median()
fare_median

14.4542

In [113]:
for dataset in all_df:
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_median #там, где пропустки, ставим медиану
    dataset.loc[np.isnan(dataset['Fare']), 'Fare'] = fare_median
    

In [114]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Создание новых переменных

In [115]:
for dataset in all_df:
    dataset["relatives"] = dataset['SibSp'] + dataset['Parch'] #супруги, братья, серстры и родители, дети

In [139]:
variables = ['Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'relatives']

In [140]:
X_train = train.loc[:, variables] #все строки и столбцы variables
y_train = train.loc[:, 'Survived']
X_test = test.loc[:, variables]

In [141]:
X_train.describe()

,Age,Pclass,SibSp,Parch,Fare,relatives
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.361582,2.308642,0.523008,0.381594,32.204208,0.904602
std,13.019697,0.836071,1.102743,0.806057,49.693429,1.613459
min,0.420000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,2.000000,0.000000,0.000000,7.910400,0.000000
50%,28.000000,3.000000,0.000000,0.000000,14.454200,0.000000
75%,35.000000,3.000000,1.000000,0.000000,31.000000,1.000000
max,80.000000,3.000000,8.000000,6.000000,512.329200,10.000000


In [142]:
les = RandomForestClassifier(n_estimators=500) #500 trees
les = les.fit(X_train, y_train)

In [143]:
y_test = les.predict(X_test)
y_test

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [144]:
y_test_df = pd.DataFrame({'PassengerId' : test['PassengerId'], 'Survived' : y_test})

In [145]:
y_test_df.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.397129
std,120.810458,0.489890
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


In [146]:
y_test_df.to_csv('forecast.csv', index=False, header=True) #false = do not want numbers of rows and colums

In [147]:
pwd()

'C:\\Users\\Максим\\Documents'

In [148]:
forecast = pd.read_csv('forecast.csv')
forecast.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


### It would be better with 'Sex' :)